In [2]:
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import plotly.express as px
from matplotlib import pyplot as plt
import math
import cv2
from skimage.draw import polygon

In [42]:
"""https://colab.research.google.com/drive/1AkBpCFU7nW_WLbeKudcLlZD-ZK5DuQCj"""
data_path = ["../data/satellite-data.csv","../data/satellite-data-6d.csv"][0]

SCALE = 100
MAX_X = 360*SCALE # longitude
MAX_Y = 180*SCALE # latitude

In [4]:
EARTH_RADIUS = 6371000 # meters
EARTH_CIRCUMFERENCE = 40075160
LAT_TO_METER = EARTH_CIRCUMFERENCE / 360
LON_TO_METER = EARTH_CIRCUMFERENCE / 360
def lat_to_meter(lat):
    return lat * LAT_TO_METER
def lon_to_meter(lon):
    return lon * LON_TO_METER
def meter_to_lat(meter):
    return (meter / LAT_TO_METER)
def meter_to_lon(meter):
    return meter / LON_TO_METER

In [67]:
def get_satellite_view_distance(satellite_fov=1, satellite_distance=500000):
    earth_radius = EARTH_RADIUS # earth radius in meters
    earth_circumference = EARTH_CIRCUMFERENCE # earth circumference in meters
    earth_max_x = earth_circumference
    earth_max_y = earth_circumference / 2
    satellite_fov_rad = satellite_fov * np.pi / 180 # satellite fov in radians
    satellite_view = 2 * np.tan(satellite_fov_rad) * satellite_distance
    scale_view = satellite_view / 5 # scale view area in meters
    grid_size = scale_view # grid size in meters
    return satellite_view
print(get_satellite_view_distance())

17455.064928217584


In [53]:
# convertions
def get_new_coordinates(point,lt,ln,angle,left=True):
    if angle < 90:
        return [point[0] - ln,point[1] + lt] if left else [point[0] + ln,point[1] - lt]
    else:
        return [point[0] - ln,point[1] - lt] if left else [point[0] + ln,point[1] + lt]

def distance_to_lat_long(distance,angle=0):
    angle = angle * np.pi / 180
    lat_dis = distance * np.sin(angle)
    lon_dis = distance * np.cos(angle)
    lat = meter_to_lat(lat_dis)
    lon = meter_to_lon(lon_dis)
    return [lon,lat]

def get_two_bound(point,lt, ln,angle):
    lon1,lat1 = get_new_coordinates(point,lt,ln,angle,left=True)
    lon2,lat2 = get_new_coordinates(point,lt,ln,angle, left=False)
    return [[lon1,lat1],[lon2,lat2]]

def get_points(distance, angle,point=(0,0),prev_point=None):
    if (prev_point is not None):
        angle = angle if point[1]>prev_point[1] else 180 - angle
        ln,lt = distance_to_lat_long(distance/2,angle)
        point1,point2 = get_two_bound(point,ln,lt,angle)
        return [point1,point2,angle]
    else:
        ln,lt = distance_to_lat_long(distance/2,angle)
        # print("ln,lt",ln,lt)
        point1,point2 = get_two_bound(point,ln,lt,angle)
        return [point1,point2,angle]

In [54]:
# test
p1,p2,ang = get_points(43000,45,[0,0])
print(p1,p2,ang)
delta_lan = p2[0] - p1[0]
delta_lat = p1[1] - p2[1]
print(delta_lan,delta_lat)
distance = math.sqrt(delta_lan**2 + delta_lat**2)
print(distance)
lat_meter = lat_to_meter(delta_lat)
lan_meter = lon_to_meter(delta_lan)
print(np.sqrt(lat_meter**2 + lan_meter**2))

[-0.1365685498544205, 0.1365685498544205] [0.1365685498544205, -0.1365685498544205] 45
0.273137099708841 0.273137099708841
0.3862741907954953
43000.0


In [10]:
def get_map(zoom=12, center=None):
    return folium.Map(zoom_start = zoom)

In [44]:
# read csv data
df = pd.read_csv(data_path)
df_columns_org = list(df.columns)
df_columns_org

df.drop(df_columns_org[3:], axis=1, inplace=True)

new_columns_names = ["Time","Latitude","Longitude"]
df.columns = new_columns_names
df.head()

,Time,Latitude,Longitude
0,15 Aug 2022 07:30:00.000,-0.000,-76.191
1,15 Aug 2022 07:31:00.000,3.137,-74.260
2,15 Aug 2022 07:32:00.000,6.269,-72.315
3,15 Aug 2022 07:33:00.000,9.391,-70.344
4,15 Aug 2022 07:34:00.000,12.498,-68.333


In [34]:
def get_point_dict(point=(0,0),angle=0,distance=0):
    return {
        "lon": point[0],
        "lat": point[1],
        "angle": angle,
        "distance": distance
    }

In [45]:
left_points = []
right_points = []
main_points = []
distance = get_satellite_view_distance()
angle = 55
prev_point = df.iloc[0]
for index, row in df.iterrows():
    point = (row[new_columns_names[2]], row[new_columns_names[1]]) # x = longitude, y = latitude
    point1,point2,_ = get_points(distance,angle,point,prev_point)
    left_points.append(get_point_dict(point1,angle,distance))
    right_points.append(get_point_dict(point2,angle,distance))
    main_points.append(get_point_dict(point,angle,distance))
    prev_point = point

In [46]:
new_map = get_map()
# draw line for each ponit group
folium.PolyLine(locations=list(map(lambda p:(p["lat"],p["lon"]),main_points)),color='red').add_to(new_map)
folium.PolyLine(locations=list(map(lambda p:(p["lat"],p["lon"]),left_points)),color='blue').add_to(new_map)
folium.PolyLine(locations=list(map(lambda p:(p["lat"],p["lon"]),right_points)),color='green').add_to(new_map)
folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(new_map)
new_map

In [55]:
earth_map_mat = cv2.Mat(np.zeros((MAX_X, MAX_Y), dtype=np.uint8))

In [56]:
# test

img = np.zeros((20, 20), dtype=np.uint8)
vertices = np.array([
    [2,6],
    [4,13],
    [15,15],
    [8,7],
])
print(vertices)
print(vertices[:,0])
rr, cc = polygon(vertices[:,0], vertices[:,1], img.shape)
print(rr,cc)
img[rr,cc] = 100

print(img.shape)

img = cv2.Mat(img)
cv2.imwrite("test_map.png",img)

[[ 2  6]
 [ 4 13]
 [15 15]
 [ 8  7]]
[ 2  4 15  8]
[ 2  3  3  3  4  4  4  4  4  4  4  5  5  5  5  5  5  5  6  6  6  6  6  6
  6  7  7  7  7  7  7  7  8  8  8  8  8  8  8  9  9  9  9  9 10 10 10 10
 10 11 11 11 11 12 12 12 13 13 14 15] [ 6  7  8  9  7  8  9 10 11 12 13  7  8  9 10 11 12 13  7  8  9 10 11 12
 13  7  8  9 10 11 12 13  7  8  9 10 11 12 13  9 10 11 12 13 10 11 12 13
 14 11 12 13 14 12 13 14 13 14 14 15]
(20, 20)


True

In [75]:
def fill_earth(df,earth):
    limit = get_satellite_view_distance()
    print("limit => ",limit)
    for i in range(df.shape[0]-1):
        point1 = [df.iloc[i][new_columns_names[2]],df.iloc[i][new_columns_names[1]]]
        point2 = [df.iloc[i+1][new_columns_names[2]],df.iloc[i+1][new_columns_names[1]]]
        point1[0] = point1[0]*SCALE+MAX_X//2
        point1[1] = point1[1]*SCALE+MAX_Y//2
        point2[0] = point2[0]*SCALE+MAX_X//2
        point2[1] = point2[1]*SCALE+MAX_Y//2
        if int(point1[1]) <= int(point2[1]):
            angle = 55
            top_1,top_2,_ = get_points(limit,angle,point2)
            top_1[0] = int(top_1[0])
            top_1[1] = int(top_1[1])
            top_2[0] = int(top_2[0])
            top_2[1] = int(top_2[1])
            bottom_1,bottom_2,_ = get_points(limit,angle,point1)
            bottom_1[0] = int(bottom_1[0])
            bottom_1[1] = int(bottom_1[1])
            bottom_2[0] = int(bottom_2[0])
            bottom_2[1] = int(bottom_2[1])
            points = np.array([top_1,bottom_1,bottom_2,top_2])
            # print("points => ",points)
            rr, cc = polygon(points[:,0], points[:,1], earth.shape)
            earth[rr,cc] += 1000
        else:
            angle = -55
            top_1,top_2,top_ang = get_points(limit,angle,point1)
            top_1[0] = int(top_1[0])
            top_1[1] = int(top_1[1])
            top_2[0] = int(top_2[0])
            top_2[1] = int(top_2[1])
            bottom_1,bottom_2,bottom_ang = get_points(limit,angle,point2)
            bottom_1[0] = int(bottom_1[0])
            bottom_1[1] = int(bottom_1[1])
            bottom_2[0] = int(bottom_2[0])
            bottom_2[1] = int(bottom_2[1])
            points = np.array([top_1,bottom_1,bottom_2,top_2])
            # print("points => ",points)
            rr, cc = polygon(points[:,0], points[:,1], earth.shape)
            earth[rr,cc] += 1000
    return earth
            
    

In [76]:
earth_map_mat = fill_earth(df,earth_map_mat)

limit =>  17455.064928217584


In [77]:
mat = cv2.Mat(np.rot90(earth_map_mat,3))
cv2.imwrite("earth_map_0.png",mat)

True